In [7]:
!nvidia-smi

Wed Sep 11 12:32:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1660 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   56C    P8               6W /  60W |      6MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
import tensorflow as tf
from helper_functions import unzip_data, create_tensorboard_callback,plot_loss_curves,compare_historys

In [9]:
unzip_data('nlp-getting-started.zip')

In [10]:
import pandas as pd

In [11]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()
train_df["text"][1]


'Forest fire near La Ronge Sask. Canada'

In [12]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)

In [13]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [14]:
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [15]:
len(train_df),len(test_df)

(7613, 3263)

In [16]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _,text,target = row
    print(f"Target:{target}","(real disaster)" if target>0 else "(not rela disaster)")
    print(f"Text :{text}")
    print('\n')
    

Target:0 (not rela disaster)
Text :Only had a crush on one girl in high school and she don't even realize it lol


Target:1 (real disaster)
Text :annihilating quarterstaff of annihilation


Target:1 (real disaster)
Text :#nar #phuket Ultimate #preparedness library: http://t.co/qYAeNDvDGC Prepare Yourself For Any Catastrophe. Over http://t.co/3Zp6Ahnsxn


Target:0 (not rela disaster)
Text :Wyrmwood: Road of the Dead (2014) was fucking awesome and it had an awesome ending too. Awesome one.


Target:1 (real disaster)
Text :New Documents Found Pointing To Japan's WWII Atomic Bomb Program http://t.co/M9mowCMVNj




In [17]:
from sklearn.model_selection import train_test_split

train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                       train_df_shuffled["target"].to_numpy(),
                                                                       test_size = 0.1,
                                                                       random_state=42)


In [18]:
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [19]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

In [20]:
from tensorflow.keras.layers import TextVectorization

In [21]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [22]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(
    max_tokens=max_vocab_length,
    standardize="lower_and_strip_punctuation",
    split = "whitespace",
    ngrams= None,
    output_mode = 'int',
    output_sequence_length = max_length,
    pad_to_max_tokens = True,
)


I0000 00:00:1726038139.050101  173716 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-11 12:32:19.051860: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [23]:
text_vectorizer.adapt(train_sentences)

In [24]:
sample_sentence = "hello world"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1400,  107,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [25]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
text_vectorizer(random_sentence)

We The Free Hailstorm Maxi http://t.co/ERWs6IELdG


<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([ 46,   2, 268, 818,   1,   1,   0,   0,   0,   0,   0,   0,   0,
         0,   0])>

In [26]:
words_in_vocabulary =  text_vectorizer.get_vocabulary()
top_5_words = words_in_vocabulary[:5]
bottom_5_words = words_in_vocabulary[-5:]
len(words_in_vocabulary),top_5_words,bottom_5_words

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

In [44]:
# from tensorflow.keras import layers

tf.random.set_seed(42)
from tensorflow.keras import layers

embeddings = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embeddings

/home/rasheek/Desktop/Github/NLP/NLP_fundamentals/tf/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<Embedding name=embedding_1, built=False>

In [46]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
print("\n")
print("embedded version")
embeddings(text_vectorizer([random_sentence]))

Why are you feeling desolate? Take the quiz: http://t.co/E9yfe3p7P1 http://t.co/8gZbGMMaa1


embedded version


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01594301,  0.04556562,  0.04478407, ..., -0.02856034,
          0.01770135, -0.03064688],
        [ 0.01443347,  0.02204731,  0.00989039, ..., -0.02060523,
          0.01328966, -0.01744287],
        [-0.01198608, -0.01785332, -0.03472489, ..., -0.0351392 ,
          0.02448305,  0.0156324 ],
        ...,
        [-0.04995975, -0.04704732,  0.02675643, ...,  0.01980003,
          0.01404873,  0.00297469],
        [-0.04995975, -0.04704732,  0.02675643, ...,  0.01980003,
          0.01404873,  0.00297469],
        [-0.04995975, -0.04704732,  0.02675643, ...,  0.01980003,
          0.01404873,  0.00297469]]], dtype=float32)>

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB()),
])

model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [48]:
baseline_score=model_0.score(val_sentences,val_labels)
baseline_score

0.7926509186351706

In [49]:
baseline_preds=model_0.predict(val_sentences)

In [50]:
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [51]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

In [52]:
def calculate_results(y_true,y_pred):
    model_accuaracy = accuracy_score(y_true,y_pred) *100
    model_precision,model_recall,model_f1,_=precision_recall_fscore_support(y_true,y_pred,average="weighted")
    model_results={
        "accuracy":model_accuaracy,
        "precision":model_precision,
        "recall":model_recall,
        "f1":model_f1
    }
    return model_results

In [53]:
baseline_results=calculate_results(val_labels,baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [54]:
from helper_functions import create_tensorboard_callback

In [55]:
SAVE_DIR='model_logs'

In [56]:
from tensorflow.keras import  layers
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embeddings(x)
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name="model_1_dense")

In [57]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization              │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [58]:
model_1.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [59]:
model_1.history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20240911-123348
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.6377 - loss: 0.6498 - val_accuracy: 0.7585 - val_loss: 0.5345
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8102 - loss: 0.4667 - val_accuracy: 0.7913 - val_loss: 0.4744
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8529 - loss: 0.3625 - val_accuracy: 0.7927 - val_loss: 0.4623
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8876 - loss: 0.2963 - val_accuracy: 0.7861 - val_loss: 0.4684
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9057 - loss: 0.2476 - val_accuracy: 0.7769 - val_loss: 0.4839


In [60]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [61]:
model_1.evaluate(val_sentences,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7670 - loss: 0.5161 


[0.48393240571022034, 0.7769029140472412]

In [63]:
model_1_pred_probs=model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


762

In [64]:
model_1_pred_probs[0]

array([0.3226549], dtype=float32)

In [65]:
model_1_pred_probs

array([[3.22654903e-01],
       [7.35205233e-01],
       [9.97719526e-01],
       [1.92684174e-01],
       [9.54333767e-02],
       [9.42622304e-01],
       [8.88863206e-01],
       [9.94247437e-01],
       [9.62057710e-01],
       [3.86765271e-01],
       [1.24990664e-01],
       [7.00662494e-01],
       [3.84656824e-02],
       [2.45965883e-01],
       [4.44994634e-03],
       [1.52278557e-01],
       [2.19577234e-02],
       [7.96897635e-02],
       [2.66841710e-01],
       [5.49795270e-01],
       [9.02879119e-01],
       [4.20814455e-02],
       [3.63316000e-01],
       [1.03577942e-01],
       [9.46989477e-01],
       [9.98866737e-01],
       [4.49176617e-02],
       [6.51680231e-02],
       [2.32563633e-02],
       [2.00772062e-01],
       [6.04766309e-01],
       [3.28063369e-01],
       [5.34305692e-01],
       [1.61876470e-01],
       [4.24877703e-01],
       [6.36627823e-02],
       [9.96055186e-01],
       [2.06018955e-01],
       [3.46896052e-02],
       [9.98727500e-01],


In [67]:
model_1_pred_probs[:10] 

array([[0.3226549 ],
       [0.73520523],
       [0.9977195 ],
       [0.19268417],
       [0.09543338],
       [0.9426223 ],
       [0.8888632 ],
       [0.99424744],
       [0.9620577 ],
       [0.38676527]], dtype=float32)

In [69]:
model_1_preds=tf.squeeze(tf.round(model_1_pred_probs))

In [70]:
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [71]:
model_1_results = calculate_results(val_labels,model_1_preds)

In [72]:
model_1_results

{'accuracy': 77.69028871391076,
 'precision': 0.7795556446582096,
 'recall': 0.7769028871391076,
 'f1': 0.7744873664507351}

In [73]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [75]:
import numpy as np 
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [76]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [77]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization              │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,840,389 (14.65 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,560,260 (9.77 MB)

In [81]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]

In [82]:
embed_weights

array([[-0.05287797, -0.054336  ,  0.01906222, ...,  0.0278998 ,
         0.00638565,  0.00641907],
       [-0.02926032,  0.03400706, -0.03001262, ...,  0.05747771,
        -0.02869362, -0.00578914],
       [-0.00966379, -0.03122672, -0.06225779, ...,  0.00447276,
        -0.02495578, -0.01201153],
       ...,
       [-0.0244567 , -0.00760514,  0.02400932, ...,  0.01903648,
         0.01111541,  0.0283663 ],
       [-0.02431114, -0.0104709 , -0.0215583 , ...,  0.06559445,
         0.00874398,  0.04094843],
       [-0.08760236, -0.07568575, -0.05625628, ...,  0.10020986,
        -0.0380353 ,  0.05622572]], dtype=float32)

In [83]:
embed_weights.shape

(10000, 128)

In [85]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()